(ar1_processes)=
# AR1 Processes

```{contents}
---
depth: 2
---
```

If you haven't already, check out our on
dynamics.

## Overview

In this lecture we are going to study a very simple class of stochastic
models called AR(1) processes.

These simple models are used again and again in economic research to
represent the dynamics of series such as

-   labor income
-   dividends
-   productivity, etc.

AR(1) processes can take negative values but are easily converted into
positive processes when necessary by a transformation such as
exponentiation.

We are going to study AR(1) processes partly because they are useful and
partly because they help us understand important concepts.

Let\'s start with some imports:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline